In [ ]:
!pip install geopandas

     |████████████████████████████████| 972kB 5.8MB/s 
     |████████████████████████████████| 6.5MB 13.1MB/s 
     |████████████████████████████████| 14.8MB 284kB/s 


In [ ]:
!pip install OSMPythonTools
!pip install -U setuptools

     |████████████████████████████████| 184kB 5.6MB/s 
  Created wheel for OSMPythonTools: filename=OSMPythonTools-0.2.9-cp36-none-any.whl size=12771 sha256=082e23b54dde62a24ed80180b2a058c2d5746179c2ca2bb42fbfb451d5913b7c
  Stored in directory: /root/.cache/pip/wheels/4e/65/34/afe0b46e3dbb1b3da010967781d4999084bfe5c63d7b2eeefd
  Created wheel for pytest-sugar: filename=pytest_sugar-0.9.4-py2.py3-none-any.whl size=8971 sha256=3d9c84c14fce7e7df8f9b8ddd3e92ee095940455818ac9c3e6da370862baf7bd
  Stored in directory: /root/.cache/pip/wheels/72/ac/7f/55050ee786fa4a2bb2d05dea0696eab826ff1d5b8a7dbd5883
Successfully built OSMPythonTools pytest-sugar
     |████████████████████████████████| 2.0MB 5.3MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 50.3.2
    Uninstalling setuptools-50.3.2:
      Successfully uninstalled setuptools-50.3.2


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import geopandas as gpd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

### Задача:

Прогноз продаж одной из популярных моделей [фичерфонов](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D1%87%D0%B5%D1%80%D1%84%D0%BE%D0%BD) (на картинке ниже пример похожего устройства) в салонах МегаФона
![](https://39.img.avito.st/640x480/8468720439.jpg)

### Исходные данные:

Датасет содержит следующие поля:

1. `point_id` - Индентификатор салона
2. `lon` - Долгота точки
3. `lat` - Широта точки
4. `target` - Значение таргета, усредненное за несколько месяцев и отнормированное

### Требования к решению и советы:

Ниже приведен список из нескольких важных пунктов, необходимых для решения задания. Выполнение каждого из пунктов влияет на итоговую оценку. Вы можете выполнить каждый из пунктов разными способами, самым лучшим будет считаться вариант, когда всё получение и обработка данных будут реализованы на Питоне (пример: вы можете скачать данные из OSM через интерфейс на сайте overpass-turbo или с помощью библиотек `overpass`/`requests`. Оба варианта будут зачтены, но больше баллов можно заработать во втором случае)



1. Салоны расположены в нескольких разных городах, вам необходимо **определить город для каждого салона** (это понадобится во многих частях задания). К этому есть разные подходы. Вы можете провести [обратное геокодирование](https://en.wikipedia.org/wiki/Reverse_geocoding) с помощью геокодера [Nominatim](https://nominatim.org/), доступного через библиотеку `geopy` примерно вот так:
```python
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)
```
В таком случае, вам придется обрабатывать полученную строку адреса, чтобы извлечь название города. Также вы можете скачать из OSM или найти в любом другом источнике границы административно территориальных границ России и пересечь с ними датасет с помощью `geopandas.sjoin` (этот вариант более надежный, но нужно будет разобраться с тем, как устроены границы АТД в OSM, обратите внимание на [этот тег](https://wiki.openstreetmap.org/wiki/Key:admin_level))


2. **Используйте данные OSM**: подумайте, какие объекты могут влиять на продажи фичерфонов. Гипотеза: такие телефоны покупают люди, приезжающие в город или страну ненадолго, чтобы вставить туда отдельную симкарту для роуминга. Можно попробовать использовать местоположения железнодорожных вокзалов (изучите [этот тег](https://wiki.openstreetmap.org/wiki/Tag:railway%3Dstation)). Необходимо использовать хотя бы 5 разных типов объектов из OSM. Скорее всего, вам придется качать данные OSM отдельно для разных городов (см. пример для Нью-Йорка из лекции)


3. **Используйте разные способы генерации признаков**: описать положение салона МегаФона относительно станций метро можно разными способами - найти ***расстояние до ближайшей станции***, или же посчитать, сколько станций попадает в ***500 метровую буферную зону*** вокруг салона. Такие признаки будут нести разную информацию. Так же попробуйте поэкспериментировать с размерами буферных зон (представьте, что значат в реальности радиусы 100, 500, 1000 метров). Попробуйте посчитать расстояние до центра города, до других объектов.

4. **Сделайте визуализации**: постройте 2-3 карты для какого нибудь из городов - как распределен в пространстве таргет, где находятся объекты, полученные вами из OSM. Можете использовать любой инструмент - обычный `plot()`, `folium`, `keplergl`. Если выберете Кеплер, обязательно сохраните в файл конфиг карты, чтобы ее можно было воспроизвести. Сделать это можно вот так:

```python
import json
json_data = kepler_map.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)
```
5. Задание не ограничено приведенными выше пунктами, попробуйте нагенерировать интересных признаков, найти в интернете дополнительные данные (в таком случае в комментарии к коду укажите ссылку на ресурс, откуда взяли данные)



6. Это довольно сложная задача - датасет очень маленький, данные по своей природе довольно случайны. Поэтому место и скор на Kaggle не будут играть решающую роль в оценке, но позволят заработать дополнительные баллы

### Read data

In [ ]:
train = pd.read_csv('data/mf_geo_train.csv')
test = pd.read_csv('data/mf_geo_test.csv')

In [ ]:
train.head(100)

,point_id,lon,lat,target
0,ommNZCUV,37.590776,55.848630,-0.348157
1,nMe2LHPb,37.784210,55.750271,1.294206
2,ZgodVRqB,39.635721,47.213330,-1.039679
3,0t2jNYdz,37.704570,55.782020,-1.169339
4,U27W4QJ7,37.643983,55.730188,-0.088837
...,...,...,...,...
95,BpoVvgKO,37.464450,55.809510,-0.650698
96,QX4SAYI9,30.247937,59.858688,-1.234169
97,5UKRuSA9,37.576824,55.838028,2.850130
98,vC4Bp1jb,30.261134,59.867868,-0.607478


In [ ]:
test.head(2)

,point_id,lon,lat,target
0,F4lXR1cG,37.681242,55.74804,0.0091
1,4LJu4GTf,60.580910,56.79586,0.0091


In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.reverse("56.79586, 60.580910")
print(location.address)

Кит, 65, улица Амундсена, Юго-Западный, Ленинский район, Екатеринбург, городской округ Екатеринбург, Свердловская область, Уральский федеральный округ, 620146, Россия


In [ ]:
train_data = train.copy()
test_data = test.copy()

In [ ]:
train_data["full_adr"] = train_data.apply(lambda row: (geolocator.reverse(str(row.lat) + ',' + str(row.lon))).address, axis=1)
test_data["full_adr"] = test_data.apply(lambda row: (geolocator.reverse(str(row.lat) + ',' + str(row.lon))).address, axis=1)

Теперь просто посмотрим на адреса, и, так как их нельзя распарсить сплитом (каждая точка имеет разное количество атрибутов и город может находиться на разных местах) и учитывая, что сет небольшой, просто посмотрим ручками, какие города есть впринципе, и составим список из них:

In [ ]:
for i in range(100):
  print(train_data["full_adr"][i])

6А, Сигнальный проезд, район Отрадное, Москва, Центральный федеральный округ, 127106, Россия
38/18, 2-я Владимирская улица, Новогиреево, район Новогиреево, Москва, Центральный федеральный округ, 4А, Россия
18, улица Зорге, ЗЖМ, Советский район, Ростов-на-Дону, городской округ Ростов-на-Дону, Ростовская область, Южный федеральный округ, 344000, Россия
Торговый центр "Серебряный Дом", 16, Большая Семёновская улица, Семёновское, район Соколиная Гора, Москва, Центральный федеральный округ, 105094, Россия
5, Кожевническая улица, район Замоскворечье, Москва, Центральный федеральный округ, 115280, Россия
12, улица 9 Мая, Северный, 3-ий микрорайон, Советский район, Красноярск, городской округ Красноярск, Красноярский край, Сибирский федеральный округ, 660000, Россия
Ледовый дворец, 1, проспект Пятилеток, округ Правобережный, Санкт-Петербург, Северо-Западный федеральный округ, 193318, Россия
Торгово-развлекательный центр "Калейдоскоп", 56, Сходненская улица, район Южное Тушино, Москва, Централь

In [ ]:
cities = ['Ростов-на-Дону', 'Красноярск',
       'Санкт-Петербург', 'Уфа', 'Москва', 'Казань', 'Екатеринбург',
       'Новосибирск', 'Нижний Новгород', 'Самара']

In [ ]:
#Fill cities column in any df
def fill_cities(df):
  for i in range(df["full_adr"].shape[0]):
    for city in cities:
      if (city in df["full_adr"][i]):
        df["city"][i] = city
        break

In [ ]:
#create city column
train_data['city'] = ""
test_data['city'] = ""

In [ ]:
fill_cities(train_data)
fill_cities(test_data)

In [ ]:
#Здесь удостоверяемся, что на тесте будут те же города
for i in range(test_data["city"].shape[0]):
  print(test_data["city"][i])

Москва
Екатеринбург
Москва
Москва
Казань
Москва
Казань
Москва
Санкт-Петербург
Санкт-Петербург
Казань
Екатеринбург
Москва
Москва
Самара
Санкт-Петербург
Москва
Москва
Новосибирск
Москва
Москва
Нижний Новгород
Москва
Новосибирск
Москва
Самара
Нижний Новгород
Москва
Ростов-на-Дону
Нижний Новгород
Санкт-Петербург
Москва
Санкт-Петербург
Самара
Новосибирск
Ростов-на-Дону
Москва
Уфа
Казань
Красноярск
Екатеринбург
Уфа
Москва
Москва
Ростов-на-Дону
Уфа
Екатеринбург
Москва
Москва
Санкт-Петербург
Екатеринбург
Санкт-Петербург
Санкт-Петербург
Самара
Уфа
Нижний Новгород
Москва
Санкт-Петербург
Москва
Самара
Новосибирск
Ростов-на-Дону
Москва
Москва
Санкт-Петербург
Москва
Москва
Новосибирск
Казань
Новосибирск
Казань
Москва
Красноярск
Москва
Москва
Казань
Ростов-на-Дону
Москва
Самара
Нижний Новгород
Москва
Санкт-Петербург
Москва
Санкт-Петербург
Санкт-Петербург
Санкт-Петербург
Москва
Москва
Москва
Москва
Санкт-Петербург
Новосибирск
Самара
Москва
Москва
Красноярск
Санкт-Петербург
Санкт-Петербург
Санкт-Петер

# Генерация признаков

Загрузим необходимые данные для городов России

In [ ]:
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

nominatim = Nominatim()
overpass = Overpass()
areaId = nominatim.query('Russia').areaId()
query = overpassQueryBuilder(area=areaId, elementType=['node'], selector='"place"~"city|town"', out='body geom;out skel qt',includeGeometry=True)
op_cities = overpass.query(query,timeout=600)

[overpass] downloading data: [timeout:600][out:json];area(3600060189)->.searchArea;(node["place"~"city|town"](area.searchArea);); out body geom;out skel qt geom;


In [ ]:
from geopy import distance

1. Расстояние до центра города

In [ ]:
print(op_cities.elements()[0].tag('name'))

Новосибирск


In [ ]:
def distance_to_center(df):
  df['center_dst'] = ""
  for row in range(len(df)):
    point_this = (df['lat'][row], df['lon'][row])
    for element in op_cities.elements():
      try:
        el_name = element.tag('name')
      except Exception:
        continue
      if df['city'][row] == el_name:
        point_center = (element.lat(), element.lon())
        df['center_dst'][row] = distance.distance(point_this, point_center).km
        break

In [ ]:
distance_to_center(train_data)
distance_to_center(test_data)

In [ ]:
train_data.head()

,point_id,lon,lat,target,full_adr,city,center_dst
0,ommNZCUV,37.590776,55.848630,-0.348157,"6А, Сигнальный проезд, район Отрадное, Москва,...",Москва,11.0593
1,nMe2LHPb,37.784210,55.750271,1.294206,"38/18, 2-я Владимирская улица, Новогиреево, ра...",Москва,10.4688
2,ZgodVRqB,39.635721,47.213330,-1.039679,"18, улица Зорге, ЗЖМ, Советский район, Ростов-...",Ростов-на-Дону,5.80346
3,0t2jNYdz,37.704570,55.782020,-1.169339,"Торговый центр ""Серебряный Дом"", 16, Большая С...",Москва,6.49854
4,U27W4QJ7,37.643983,55.730188,-0.088837,"5, Кожевническая улица, район Замоскворечье, М...",Москва,2.80273


Если посмотреть несколько точек вручную, то расстояния считаются правильно (метры не счиатем)

2. Общепит в радиусе 1 км (количество точек)

In [ ]:
import math

In [ ]:
# degrees to radians
def deg2rad(degrees):
    return math.pi*degrees/180.0
# radians to degrees
def rad2deg(radians):
    return 180.0*radians/math.pi

# Semi-axes of WGS-84 geoidal reference
WGS84_a = 6378137.0  # Major semiaxis [m]
WGS84_b = 6356752.3  # Minor semiaxis [m]

# Earth radius at a given latitude, according to the WGS-84 ellipsoid [m]
def WGS84EarthRadius(lat):
    An = WGS84_a*WGS84_a * math.cos(lat)
    Bn = WGS84_b*WGS84_b * math.sin(lat)
    Ad = WGS84_a * math.cos(lat)
    Bd = WGS84_b * math.sin(lat)
    return math.sqrt( (An*An + Bn*Bn)/(Ad*Ad + Bd*Bd) )

# Bounding box surrounding the point at given coordinates,
# assuming local approximation of Earth surface as a sphere
# of radius given by WGS84
def boundingBox(latitudeInDegrees, longitudeInDegrees, halfSideInKm):
    lat = deg2rad(latitudeInDegrees)
    lon = deg2rad(longitudeInDegrees)
    halfSide = 1000*halfSideInKm

    # Radius of Earth at given latitude
    radius = WGS84EarthRadius(lat)
    # Radius of the parallel at given latitude
    pradius = radius*math.cos(lat)

    latMin = lat - halfSide/radius
    latMax = lat + halfSide/radius
    lonMin = lon - halfSide/pradius
    lonMax = lon + halfSide/pradius

    return [rad2deg(latMin), rad2deg(lonMin), rad2deg(latMax), rad2deg(lonMax)]

In [ ]:
boundingBox(55.848630, 37.590776, 0.5)

[55.844128113844675, 37.58275669644521, 55.85313188615533, 37.59879530355479]

In [ ]:
def count_cafes(df, radius_in_km):
  df['cafes'+str(radius_in_km)] = ""
  for row in range(len(df)):
    bbox = boundingBox(df['lat'][row], df['lon'][row], radius_in_km/2)
    query = overpassQueryBuilder(bbox=bbox, elementType='node', selector='"amenity"~"fast_food|cafe"', out='body geom;out skel qt',includeGeometry=True)
    cafes = overpass.query(query,timeout=600)
    number_of_cafes = len(cafes.elements())
    df['cafes'+str(radius_in_km)][row] = number_of_cafes


In [ ]:
count_cafes(train_data, 1)
count_cafes(test_data, 1)

[overpass] downloading data: [timeout:600][out:json];(node["amenity"~"fast_food|cafe"](55.844128113844675,37.58275669644521,55.853131886155346,37.59879530355479);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["amenity"~"fast_food|cafe"](55.74576913806643,37.77621097052211,55.75477286193357,37.79220902947789);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["amenity"~"fast_food|cafe"](47.20883032957939,39.629096731938475,47.217829670420606,39.64234526806152);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["amenity"~"fast_food|cafe"](55.77751813024447,37.6965644400763,55.78652186975553,37.7125755599237);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["amenity"~"fast_food|cafe"](55.72568614301598,37.635988094640155,55.73468985698401,37.65197790535984);); out body geom;out skel qt geom;
[overpass] downloading

In [ ]:
train_data.head()

,point_id,lon,lat,target,full_adr,city,center_dst,cafes1,cafes
0,ommNZCUV,37.590776,55.848630,-0.348157,"6А, Сигнальный проезд, район Отрадное, Москва,...",Москва,11.0593,18,
1,nMe2LHPb,37.784210,55.750271,1.294206,"38/18, 2-я Владимирская улица, Новогиреево, ра...",Москва,10.4688,38,
2,ZgodVRqB,39.635721,47.213330,-1.039679,"18, улица Зорге, ЗЖМ, Советский район, Ростов-...",Ростов-на-Дону,5.80346,8,
3,0t2jNYdz,37.704570,55.782020,-1.169339,"Торговый центр ""Серебряный Дом"", 16, Большая С...",Москва,6.49854,68,
4,U27W4QJ7,37.643983,55.730188,-0.088837,"5, Кожевническая улица, район Замоскворечье, М...",Москва,2.80273,124,


Судя по расположению, всё верно (в центре Москвы сильно больше кафе-фастфудов, чем в Отрадном)

3. Остановки транспорта

In [ ]:
def count_smth(df, column_name, radius_in_km, selector):
  df[column_name+str(radius_in_km)] = ""
  for row in range(len(df)):
    bbox = boundingBox(df['lat'][row], df['lon'][row], radius_in_km/2)
    query = overpassQueryBuilder(bbox=bbox, elementType='node', selector=selector, out='body geom;out skel qt',includeGeometry=True)
    smth = overpass.query(query,timeout=600)
    number_of_smth = len(smth.elements())
    df[column_name+str(radius_in_km)][row] = number_of_smth


In [ ]:
selector = '"public_transport"~"platform|stop_position|station|stop_area"'
count_smth(train_data, 'transport', 1, selector)
count_smth(test_data, 'transport', 1, selector)

[overpass] downloading data: [timeout:600][out:json];(node["public_transport"~"platform|stop_position|station|stop_area"](55.844128113844675,37.58275669644521,55.853131886155346,37.59879530355479);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["public_transport"~"platform|stop_position|station|stop_area"](55.74576913806643,37.77621097052211,55.75477286193357,37.79220902947789);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["public_transport"~"platform|stop_position|station|stop_area"](47.20883032957939,39.629096731938475,47.217829670420606,39.64234526806152);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["public_transport"~"platform|stop_position|station|stop_area"](55.77751813024447,37.6965644400763,55.78652186975553,37.7125755599237);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["public_transport"~

4, 5. Электроника и офисы

In [ ]:
selector_electronics_shops = '"shop"~"computer|electronics|mobile_phone|radiotechnics"'
count_smth(train_data, 'electronics', 1, selector_electronics_shops)
count_smth(test_data, 'electronics', 1, selector_electronics_shops)

selector_offices = '"office"~"accountant|company|coworking"'
count_smth(train_data, 'offices', 1, selector_offices)
count_smth(test_data, 'offices', 1, selector_offices)

[overpass] downloading data: [timeout:600][out:json];(node["shop"~"computer|electronics|mobile_phone|radiotechnics"](55.65975515927426,37.4613315971626,55.66875884072573,37.4772944028374);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["shop"~"computer|electronics|mobile_phone|radiotechnics"](55.78216112910085,49.101598483960984,55.79116487089914,49.11761151603902);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["shop"~"computer|electronics|mobile_phone|radiotechnics"](56.25485601305012,43.84886865621848,56.26385998694986,43.86507934378153);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["shop"~"computer|electronics|mobile_phone|radiotechnics"](59.99321712494781,29.75653487056141,60.0022228750522,29.77454512943859);); out body geom;out skel qt geom;
[overpass] downloading data: [timeout:600][out:json];(node["shop"~"computer|electronics|mobile_phone|r

In [ ]:
train_data.drop('full_adr', inplace=True, axis=1)
test_data.drop('full_adr', inplace=True, axis=1)

In [ ]:
train_data_new = train_data.drop('city', axis=1)
test_data_new = test_data.drop('city', axis=1)

In [ ]:
train_data_new.drop([313], axis=0, inplace=True)

### Fit model

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data_new.drop(['target', 'point_id'], axis=1), train_data_new[['target']])
model = LinearRegression().fit(X_train, y_train)

In [ ]:
X_train

,lon,lat,center_dst,cafes1,transport1,electronics1,offices1
12,55.993548,54.713414,3.28822,6,14,4,2
423,37.752130,55.619640,16.8466,10,26,4,0
22,30.348665,59.832466,11.9776,54,64,22,4
383,39.756131,47.282688,7.60945,0,4,0,2
257,43.877078,56.261653,10.8072,4,28,2,0
...,...,...,...,...,...,...,...
266,37.514509,55.744566,6.50042,22,22,2,4
268,37.564184,55.678416,8.69142,40,42,6,4
94,56.075499,54.817080,13.0515,0,22,2,0
270,30.470336,59.865059,11.9061,20,76,2,2


In [ ]:
X_valid.drop([313], inplace=True, axis=0)

In [ ]:
y_valid.drop([313], inplace=True, axis=0)

In [ ]:
mean_absolute_error(y_valid, model.predict(X_valid))

0.6816327469435615

In [ ]:
test_data_new

,point_id,lon,lat,target,center_dst,cafes1,transport1,electronics1,offices1
0,F4lXR1cG,37.681242,55.748040,0.0091,4.01206,64,58,20,4
1,4LJu4GTf,60.580910,56.795860,0.0091,5.09674,12,28,4,0
2,kLuAAN3s,37.598614,55.781357,0.0091,3.63988,232,80,16,6
3,OxQHvaNu,37.794051,55.717468,0.0091,11.6833,50,36,20,2
4,paQsTa1K,49.213026,55.748290,0.0091,6.74192,12,26,10,0
...,...,...,...,...,...,...,...,...,...
102,y8oQuX5v,30.353777,60.049792,0.0091,12.5496,18,66,10,2
103,4nmfqUw0,92.928927,56.116262,0.0091,12.4388,2,8,4,2
104,N9O45mAh,93.015993,56.023697,0.0091,9.09468,2,32,4,0
105,h2InCLKa,30.381172,59.871149,0.0091,8.36064,30,86,10,0


In [ ]:
subm = pd.DataFrame(test_data_new['point_id'])
subm['target'] = model.predict(test_data_new.drop(['point_id', 'target'], axis=1))
subm.to_csv('subm.csv', encoding = 'utf-8', index=False)

### Make submission

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')
submission['target'] = model.predict(X_valid.drop('point_id', axis=1))
submission.to_csv('data/my_submission_01.csv', index=False)